#### Import das bibliotecas

In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler

#### Leitura do Arquivo Excel

In [110]:
dataframe = pd.read_excel('Base policia.xlsx')

In [112]:
dataframe['class'] = dataframe['class'].apply(lambda x: 'Ano de Eleição' if x == 1 else 'Ano Comum')
# .value_counts()

In [115]:
dataframe['class'].value_counts()

Ano Comum         18383
Ano de Eleição     4802
Name: class, dtype: int64

In [116]:
df = dataframe

In [117]:
df.columns

Index(['ID', 'class', 'CISP', 'area_cisp', 'regiao', 'hom_doloso',
       'lesao_corp_morte', 'latrocinio', 'tentat_hom', 'lesao_corp_dolosa',
       'estupro', 'hom_culposo', 'lesao_corp_culposa', 'encontro_cadaver',
       'encontro_ossada', 'roubo_comercio', 'roubo_residencia',
       'roubo_veiculo', 'roubo_carga', 'roubo_transeunte', 'roubo_em_coletivo',
       'roubo_banco', 'roubo_cx_eletronico', 'roubo_celular',
       'roubo_conducao_saque', 'roubo_bicicleta', 'outros_roubos',
       'furto_veiculos', 'furto_bicicleta', 'outros_furtos', 'sequestro',
       'extorsao', 'sequestro_relampago', 'estelionato', 'apreensao_drogas',
       'cump_mandado_prisao', 'ameaca', 'pessoas_desaparecidas',
       'hom_por_interv_policial', 'armas_apreendidas',
       'pol_militares_mortos_serv', 'pol_civis_mortos_serv',
       'indicador_letalidade'],
      dtype='object')

In [118]:
prepara = preparBase(df)
prepara.normaliza(columnIgnoring=['class','ID'],normalize=False)

C:\ProgramData\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by M

#### Contagem de Registros em cada Classe

In [119]:
df.groupby('class').count()[['CISP']]

,CISP
class,
Ano Comum,18383
Ano de Eleição,4802


### Divisão da base de dados em Treino-Teste

In [127]:
X = prepara.df_norm.drop('class',axis=1)
y = prepara.df_norm['class']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.47, random_state=4)

#### Separação da base em Treino e Teste pelo método de Cross-Validation

In [10]:
# from sklearn.model_selection import KFold
# kf = KFold(n_splits=10, shuffle=True)

In [8]:
# for train_index, test_index in kf.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X.loc[train_index], X.loc[test_index]
#     y_train, y_test = y.loc[train_index], y.loc[test_index]
    
#     svc_model = SVC()
#     svc_model.fit(X_train,y_train)
    
#     predictions = svc_model.predict(X_test)
#     print(confusion_matrix(y_test,predictions))

### Treinamento do Modelo

https://www.youtube.com/watch?v=6dbrR-WymjI

In [128]:
from sklearn.svm import SVC
svc_model = SVC()
svc_model.fit(X_train,y_train)

C:\ProgramData\Miniconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

### Avaliação do Modelo

In [129]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
predictions = svc_model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

[[8581    1]
 [   2 2313]]
                precision    recall  f1-score   support

     Ano Comum       1.00      1.00      1.00      8582
Ano de Eleição       1.00      1.00      1.00      2315

     micro avg       1.00      1.00      1.00     10897
     macro avg       1.00      1.00      1.00     10897
  weighted avg       1.00      1.00      1.00     10897

0.9997246948701477


In [57]:
class preparBase():
    def __init__(self, df):
        self.df = df
    
    def normaliza(self, columnIgnoring:list, normalize:bool, columnDelete:list=[]):
        scaler = MinMaxScaler()
        df = self.df
        df_norm = pd.DataFrame()
        for col in df.columns:
            if(col in columnDelete):            
                continue
                
            if(col in columnIgnoring or normalize):
                df_norm[col] = df[col]
                
            else:
                col_zscore = col + '_zscore'
                df_norm[col_zscore] = scaler.fit_transform(df[[col]])
        self.df_norm = df_norm

#### Exportação da Base de Dados Normalizada

In [130]:
prepara.df_norm.to_csv('policia_records_normalizada.csv', index=False)

In [131]:
df.to_csv('policia_records.csv', index=False)